In [ ]:
import os
import glob
import tempfile
import re

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

from geotopy import GEOtop

sns.set(rc={'figure.figsize': (15, 10), 'figure.dpi' : 250})

In [ ]:
def run_geotop():
    with tempfile.TemporaryDirectory() as tmpdir:

        model = GEOtop('data/inputs', working_dir=tmpdir, exe='../../geotop/build/geotop')

        print(f'Running GEOtop...')
        model.run(check=True, capture_output=True)

        print('Saving dataframes...')
        glob_path = os.path.join(tmpdir, 'output/*.txt')
        for file_path in glob.iglob(glob_path):
            df = pd.read_csv(file_path, na_values=['-9999'])
            df.rename(columns={'Date12[DDMMYYYYhhmm]': 'Date'}, inplace=True)
            basename = os.path.basename(file_path)
            basename = basename.split('.')[0]
            key = re.match(r'[a-z]+', basename).group(0)
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
            df.to_hdf('data/Matsch_B2.h5', key=key)
        
        
        obs = pd.read_csv('data/inputs/obs.csv', na_values=['-9999', '-99.99'])
        obs.rename(columns={'Date12.DDMMYYYYhhmm.': 'Date'}, inplace=True)
        obs['Date'] = pd.to_datetime(obs['Date'])
        obs.set_index('Date', inplace=True)
        obs.to_hdf('data/Matsch_B2.h5', 'obs')

In [ ]:
run_geotop()

print('Reading dataframes...')
obs = pd.read_hdf('data/Matsch_B2.h5', 'obs')
point = pd.read_hdf('data/Matsch_B2.h5', 'point')
liq = pd.read_hdf('data/Matsch_B2.h5', 'thetaliq')
ice = pd.read_hdf('data/Matsch_B2.h5', 'thetaliq')

sim = pd.concat([point, liq + ice], axis=1)


In [ ]:
def compare(observations, simulation, periods=None, name=None, unit=None, cum=False, rel=False):
    
    if not periods:
        periods = {'Daily': 'D', 'Weekly': 'W', 'Monthly': 'M'}
    
    fig, axes = plt.subplots(ncols=3, 
                             nrows=len(periods), 
                             constrained_layout=True)
    
    if name:
        fig.suptitle(f'{name} at different time scales')
    
    for i, (Tstr, T) in enumerate(periods.items()):
        comp_plot, diff_plot, hist_plot = axes[i, :]
        
        if cum:
            obs_resampled = observations.resample(T).sum()
            sim_resampled = simulation.resample(T).sum()
        else:
            obs_resampled = observations.resample(T).mean()
            sim_resampled = simulation.resample(T).mean()
        
        err = obs_resampled - sim_resampled        
        if rel:
            err = err / obs_resampled.abs()
        
        data = pd.DataFrame({'Observations': obs_resampled, 'Simulation': sim_resampled})
        sns.lineplot(data=data, ax=comp_plot)
        comp_plot.set_title(Tstr)
        comp_plot.set_xlabel("")
        if unit:
            comp_plot.set_ylabel(f'[{unit}]')
        
        sns.lineplot(data=err, ax=diff_plot)
        plt.setp(diff_plot.get_xticklabels(), rotation=20)
        if rel:
            diff_plot.set_ylabel(f'Relative error')
            diff_plot.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
        elif unit:
            diff_plot.set_ylabel(f'Error [{unit}]')
        else:
            diff_plot.set_ylabel(f'Error')
        
        sns.distplot(err, rug=True, vertical=True, hist=True, ax=hist_plot)
        y1, y2 = diff_plot.get_ylim()
        hist_plot.set_ylim(y1,y2)
        hist_plot.set_yticklabels([])
    return fig

In [ ]:
meteo = pd.read_csv("data/inputs/meteo0001.txt", na_values=['-9999'])
meteo['Date'] = pd.to_datetime(meteo['Date'])
meteo.set_index('Date', inplace=True)

## Air Temperature (sim vs meteo)

In [ ]:
try:
    compare(sim['Tair[C]'], 
            meteo['AirT'], 
            name='Air temperature (sim vs meteo)', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Air temperature (obs vs meteo)

In [ ]:
try:
    compare(obs['air_temperature'], 
            meteo['AirT'], 
            name='Air temperature (obs vs meteo)', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Air Temperature (obs vs sim)

In [ ]:
try:
    compare(obs['air_temperature'], 
            sim['Tair[C]'], 
            name='Air temperature', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Air Temperature (sim vs meteo, +1 shift)

In [ ]:
try:
    compare(sim['Tair[C]'].shift(1), 
            meteo['AirT'], 
            name='Air temperature (sim vs meteo)', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Shift estimation

In [ ]:
x = {}
for n in range(-48, 48):
    sim_ = sim.shift(n)
    T = sim_['Tair[C]'] - obs['air_temperature']
    R = (sim_['Prain_over_canopy[mm]'] + sim_['Psnow_over_canopy[mm]']) - obs['rainfall_amount']
    W = sim_['Wind_speed[m/s]'] - obs['wind_speed']
    H = sim_['Relative_Humidity[-]'] - obs['relative_humidity']
    s = (T.abs() / T.abs().max()) + (R.abs() / R.abs().max()) + (W.abs() / W.abs().max()) + (H.abs() / H.abs().max())
    s = s.mean()
    x[n] = s

In [ ]:
n = min(x, key=x.get)
sim = sim.shift(n)
print("Estimated shift: ", n)

## Air Temperature (sim vs meteo)

In [ ]:
try:
    compare(sim['Tair[C]'], 
            meteo['AirT'], 
            name='Air temperature (sim vs meteo)', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Air Temperature

In [ ]:
try:
    compare(obs['air_temperature'], 
            sim['Tair[C]'], 
            name='Air temperature', 
            unit='C',
            rel=False)
    plt.show()
except:
    pass

## Rainfall Amount

In [ ]:
try:
    compare(obs['rainfall_amount'], 
            sim['Prain_over_canopy[mm]'] + sim['Psnow_over_canopy[mm]'],
            name='Rainfall amount',
            unit='mm',
            rel=False,
            cum=True)
    plt.show()
except:
    pass

## Relative Humidity

In [ ]:
try:
    compare(obs['relative_humidity'], 
            sim['Relative_Humidity[-]'], 
            name='Relative humidity')
    plt.show()
except:
    pass

## Wind Speed

In [ ]:
try:
    compare(obs['wind_speed'], 
            sim['Wind_speed[m/s]'], 
            name='Wind speed', 
            unit='m/s')
    plt.show()
except:
    pass

## Upwelling Shortwave Flux

In [ ]:
try:
    compare(obs['surface_upwelling_shortwave_flux'], 
            sim['SWup[W/m2]'], 
            name='Upwelling shortwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Upwelling Longwave Flux

In [ ]:
try:
    compare(obs['surface_upwelling_longwave_flux'], 
            sim['LWup[W/m2]'], 
            name='Upwelling longwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Downwelling Shortwave Flux

In [ ]:
try:
    compare(obs['surface_downwelling_shortwave_flux'], 
            sim['SWin[W/m2]'],
            name='Downwelling shortwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Downwelling Longwave Flux

In [ ]:
try:
    compare(obs['surface_downwelling_longwave_flux'], 
            sim['LWin[W/m2]'], 
            name='Downwelling longwave flux', 
            unit='W/m²')
    plt.show()
except:
    pass

## Net Radiation

In [ ]:
try:
    compare(obs['net_radiation'], 
            sim['LWin[W/m2]'] + point['SWin[W/m2]'] - point['LWup[W/m2]'] - point['SWup[W/m2]'],
            name='Net radiation', 
            unit='W/m²')
    plt.show()
except:
    pass

## Latent  Heat Flux in Air

In [ ]:
try:
    compare(obs['latent_heat_flux_in_air'], 
            sim['Canopy_fraction[-]'] * (sim['LEg_veg[W/m2]'] + sim['LEv[W/m2]'] 
                                           - sim['LEg_unveg[W/m2]']) 
            + sim['LEg_unveg[W/m2]'],
            name='Latent heat flux in air', 
            unit='W/m²')
    plt.show()
except:
    pass

## Sensible Heat Flux in Air

In [ ]:
try:
    compare(obs['sensible_heat_flux_in_air'], 
            sim['Canopy_fraction[-]'] * (sim['Hg_veg[W/m2]'] + sim['Hv[W/m2]'] - sim['Hg_unveg[W/m2]']) +
            sim['Hg_unveg[W/m2]'],
            name='Sensible flux in air',
            unit='W/m²')
    plt.show()
except:
    pass

## Soil Moisture Content 50

In [ ]:
try:
    compare(obs['soil_moisture_content_50'], 
            sim['50.000000 '],
            name='Soil moisture content 50')
    plt.show()
except:
    pass

## Soil Moisture Content 200

In [ ]:
try:
    compare(obs['soil_moisture_content_200'], 
            sim['150.000000 '],
            name='Soil moisture content 200')
    plt.show()
except:
    pass